# Imports

In [1]:
import sys
import json
import pickle
import random
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout

Using TensorFlow backend.


# Write song

In [ ]:
'''Loads in indices file'''

char_indices = pickle.load(open("char_indic_100.json", "rb"))
indices_char = pickle.load(open("indic_char_100.json", "rb"))

In [ ]:
'''Creates model and loads in weights'''

maxlen = 20

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(char_indices))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(char_indices)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.load_weights("weights.h5")

In [ ]:
'''Hhelper function to sample an index from a probability array'''

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
'''Input seed and generate song'''

text = "one dark and stormy night" #this is the seed that you input to start the song-maker
generated = ''
start_index = 0
sentence = text[start_index: start_index + maxlen]
generated += sentence


for i in range(1000):
    x = np.zeros((1, maxlen, len(char_indices)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    diversity = 1.0
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]
    
    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()